# Powers of Matrices

The `numpy.linalg` library contains function `matrix_power` that on input `(M,k)` such that `M` is a square matrix and `k` is an integer computes `M` raised to the power `k`, i.e$.$ `M` multiplied with itself `k` times. We will develop in two ways our own version of the `matrix_power` function for the case when `k` is a non-negative integer. Firstly we will develop a simple brute force function in which you carry out matrix multiplication `k` times. Secondly we will develop a clever function that computes `M` raised to the power `k` very efficiently (at least for large `k`) provided that `M` is diagonalisable.

**-** Given $n \times n$ matrix $M$, $M^0 = I$ where $I$ is the $n \times n$ identity matrix. Also of course $M^1 = M$. <br />

**-** Matrix $M$ is diagonalisable if the set of eigenvectors $\{\mathbf{v}_1,\dots,\mathbf{v}_n\}$ of $M$ is linearly independent. <br />

**-** An integer $k$ is *non-negative* if $k \ge 0$.

This function `brute_matrix_power` will take as inputs a square `numpy` matrix `M` and a non-negative integer `k` and computes and returns `M` raised to the power `k`.  This function will deal with the case  `k` = `0` appropriately. For matrix multiplication we will use the `dot` function from the `numpy` library. We are further assuming that input `M` is a square matrix and input `k` is a non-negative integer. I.e. no need to deal with exceptions. 

In [2]:
import numpy as np
import numpy.linalg as lag

In [3]:
def brute_matrix_power(M,k):
        shape = np.shape(M)
        d = shape[0]
        if k == 0:
            m = [[0 for x in range(d)] for y in range(d)]
            for i in range(0,d):
                m[i][i] = 1
            m = np.array(m)
            return m 
        elif k == 1:
            return M
        else:
            mat = M
            for z in range(k-1):
                l = []
                for j in range(d):
                    col = []
                    for k in range(d):
                        e = []
                        for i in range(d):
                            e.append(mat[i,k]*M[j,i])
                        f = sum(e)
                        col.append(f)
                    l.append(col)
                mat = np.array(l)
            return mat

In [4]:
# inital tests
M1 = np.array([[1.0,2.0],[3.0,4.0]])
M1_exp = brute_matrix_power(M1,7)
M2 = np.array([[0.92740639, 0.25351063, 0.24551659],
               [0.98468568, 0.17874981, 0.95155668],
               [0.17727058, 0.79925759, 0.90888965]])
M2_exp = brute_matrix_power(M2,19)
assert np.allclose(M1_exp, lag.matrix_power(M1,7))
assert np.allclose(M2_exp, lag.matrix_power(M2,19))

A matrix $D = (d_{ij})$ is *diagonal* if it is of the form

 $$                                                                                                               
    \begin{pmatrix}                                                                           
      d_{11} & 0 & \cdots  & 0 \\                                                             
      0  & d_{22} & \cdots & 0  \\                                                             
      \vdots & \vdots & \ddots  & \vdots \\                                                   
      0 & 0 & \cdots  & d_{nn}                                                                 
    \end{pmatrix} \,,                                                                                              
    $$    
i.e. if the only non-zero components of $D$ lie on the diagonal. Now, it is  is easy to see that, for any non-negative integer $k$, 

 $$
 D^k \;=\;
    \begin{pmatrix}                                                                           
      d_{11}^k & 0 & \cdots  & 0 \\                                                                 0  & d_{22}^k & \cdots & 0  \\                                                           
      \vdots & \vdots & \ddots  & \vdots \\                                                   
      0 & 0 & \cdots  & d_{nn}^k                                                               
    \end{pmatrix} \, ,                                                                                             
$$

i.e. that $D^k$ is the diagonal matrix whose diagonal components correspond to the diagonal components of $D$ raised to the power $k$. We will use this idea to write a function `diag_exp` that takes as inputs a diagonal `numpy` matrix `D` and a non-negative integer `k` and computes and returns the matrix `D` raised to the power `k`. This function `diag_exp` will not carry out any matrix multiplication. It will also not carry out any matrix exponentiation.

We will assume that input `D` is a diagonal matrix and input `k` is a non-negative integer. 

In [5]:
def diag_exp(D,k):
    takeout = []
    shape = np.shape(D)
    d = shape[0]
    for i in range(d):
        takeout.append(D[i,i])
    for j in range(d):
        takeout[j] = takeout[j]**k
    m = [[0 for x in range(d)] for y in range(d)]
    for i in range(0,d):
        m[i][i] = takeout[i]
    m = np.array(m)
    return m

In [6]:
# tests:
D1 = np.diag([1.0,2.0,3.0])
D1_zero = diag_exp(D1,0)
D1_exp = diag_exp(D1,12)
D2 = np.diag([4.08801819, 7.62779907, 7.43214746, 3.62364886])
D2_exp = diag_exp(D2,6)
assert np.allclose(D1_zero,np.eye(3)) 
assert np.allclose(D1_exp,lag.matrix_power(D1,12))
assert np.allclose(D2_exp,lag.matrix_power(D2,6))

Suppose that $M$ is an  $n \times n $ matrix which  is diagonalisable with eigenvalues $\lambda_1,\dots,\lambda_n$ and eigenvectors $\mathbf{v}_1,\dots,\mathbf{v}_n$ such that $M \mathbf{v}_j = \lambda_j \mathbf{v}_j$ for $j = 1,\dots,n$. 
Then we know that $M = NDN^{-1}$ where $D = (d_{ij})$ is the diagonal matrix whose diagonal components are the eigenvalues of $M$  and $N$ is a  $n \times n$  matrix whose columns are the eigenvectors of $M$. Accordingly for each $j = 1,\dots, n$, we have that $d_{jj} = \lambda_j$  and the  $j$th column of $N$ is the eigenvector  $\mathbf{v}_j$ associated with eigenvalue $\lambda_j$. Notice also that 

$$
M^2 = NDN^{-1}NDN^{-1} = ND^2N^{-1}
$$ 

so that we can easily see by induction that $M^k = ND^kN^{-1}$ for *any* integer $k \ge 0$. With this in mind we will write a function `d_matrix_exp` that takes as input a diagonalisable `numpy` matrix `M` and a non-negative integer `k` and  computes and returns `M` to the power `k`.

1. The function `d_matrix_exp` will carry out only two matrix multiplication operations. 
2. In the definition of the function we will use the function `diag_exp` defined above.  

In [7]:
def d_matrix_exp(M,k):
    eig_values, eig_vectors = lag.eig(M)
    N = eig_vectors
    v = len(eig_vectors)
    Ninv = lag.inv(N)
    empty_rix = [([0]*v) for i in range(v)]
    for i in range(v):
        empty_rix[i][i] = eig_values[i]
    ello = np.array(empty_rix)
    D = diag_exp(ello,k)
    a = np.dot(N,D)
    b = np.dot(a,Ninv)
    return b

In [8]:
# Tests
E1 = np.array([[0.28106291, 0.51904762, 0.64148366, 0.34731372],
               [0.26979199, 0.26259106, 0.06760985, 0.37285164],
               [0.52170856, 0.06242276, 0.48040518, 0.05721432],
               [0.46227145, 0.61229525, 0.36713064, 0.72517594]])
E1_zero = d_matrix_exp(E1,0)
E1_exp = d_matrix_exp(E1,10)